In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
from os import makedirs
from os.path import join, exists, expanduser
from keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications.imagenet_utils import decode_predictions

In [6]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [7]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15217869267425815957
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 7806648320
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 3077089086640572423
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:0a:00.0, compute capability: 8.6"
 xla_global_id: 416903419]

In [8]:
img_des = pd.read_csv('D:\RMBI_MMKG_data\img_des.csv')
img_des

,LabelName,name,description
0,/m/027rn,Dominican Republic,The Dominican Republic is a country located on...
1,/m/017dcd,Mighty Morphin Power Rangers,Mighty Morphin Power Rangers is an American su...
2,/m/01sl1q,Michelle Rodriguez,Mayte Michelle Rodriguez is an American actres...
3,/m/0cnk2q,Australia national football team,The Australia men's national soccer team repre...
4,/m/02_j1w,Defender,"In the sport of association football, a defend..."
...,...,...,...
14179,/m/0dlj8q2,Le Monde's 100 Books of the Century,The 100 Books of the Century is a list of the ...
14180,/m/05zp8,Palace,"A palace is a grand residence, especially a ro..."
14181,/m/024030,Padma Vibhushan,The Padma Vibhushan is the second-highest civi...
14182,/m/02y_9cf,Ethiopian Empire,"The Ethiopian Empire, also formerly known by t..."


In [9]:
SIZE = 1000

In [10]:
img_des_trunc = img_des.head(SIZE)
img_des_trunc

,LabelName,name,description
0,/m/027rn,Dominican Republic,The Dominican Republic is a country located on...
1,/m/017dcd,Mighty Morphin Power Rangers,Mighty Morphin Power Rangers is an American su...
2,/m/01sl1q,Michelle Rodriguez,Mayte Michelle Rodriguez is an American actres...
3,/m/0cnk2q,Australia national football team,The Australia men's national soccer team repre...
4,/m/02_j1w,Defender,"In the sport of association football, a defend..."
...,...,...,...
995,/m/01b9ck,Darryl F. Zanuck,Darryl Francis Zanuck was an American film pro...
996,/m/011ywj,Gosford Park,Gosford Park is a 2001 satirical black comedy ...
997,/m/0325pb,Alpha Delta Pi,"Alpha Delta Pi, commonly known as ADPi, is an ..."
998,/m/0mnrb,Henrico County,"Henrico County, officially the County of Henri..."


In [11]:
y = []
x_img = []
import os

for i in range(SIZE):
    path_of_the_directory = os.path.join('D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader', img_des['name'][i])
    ext = ('.jpg')
    for path, dirc, files in os.walk(path_of_the_directory):
        for name in files:
            if name.endswith(ext):
                
                img_path = os.path.join(path, name)
                try:                
                    img = image.load_img(img_path, target_size=(224, 224))
                    img = image.img_to_array(img)
                
                    x_img.append(img)
                    y.append(img_des['name'][i])
                except:
                    print(img_path)

C:\Users\Alex\anaconda3\envs\gpu\lib\site-packages\PIL\TiffImagePlugin.py:845: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 2. 
  warnings.warn(str(msg))


D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\World War II\img_1460.jpg
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Crystal Palace F.C.\img_2403.jpg
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Crystal Palace F.C.\img_2404.jpg
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\United Nations\img_3556.jpg
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Marvel Entertainment\img_4278.jpg
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Billy Connolly\img_5103.jpg
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\APOEL FC\img_5873.jpg
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Fayetteville\img_104019.jpg
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\SG Dynamo Dresden\img_6375.jpg
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Shrek\img_6777.jpg
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Dan Povenmire\img_7498.jpg
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Election\img_57947.jpg


In [12]:
y = np.array(y)
x_img = np.array(x_img)

In [13]:
print(y.shape)
print(x_img.shape)

(8493,)
(8493, 224, 224, 3)


In [14]:
y

array(['Dominican Republic', 'Dominican Republic', 'Dominican Republic',
       ..., 'Theatrical producer', 'Theatrical producer',
       'Theatrical producer'], dtype='<U97')

In [17]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y)
y_encoded = le.transform(y)

In [18]:
y_encoded

array([265, 265, 265, ..., 890, 890, 890])

In [19]:
from tensorflow.keras.utils import to_categorical
y_one_hot = to_categorical(y_encoded)
y_one_hot

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [20]:
y_one_hot.shape

(8493, 983)

In [21]:
from sklearn.model_selection import train_test_split
x_img_train, x_img_test, y_one_hot_train, y_one_hot_test, y_encoded_train, y_encoded_test = train_test_split(x_img, y_one_hot, y_encoded, test_size=0.2)

In [22]:
x_img_train

array([[[[255., 255., 255.],
         [255., 255., 255.],
         [255., 255., 255.],
         ...,
         [255., 255., 255.],
         [255., 255., 255.],
         [255., 255., 255.]],

        [[255., 255., 255.],
         [255., 255., 255.],
         [255., 255., 255.],
         ...,
         [255., 255., 255.],
         [255., 255., 255.],
         [255., 255., 255.]],

        [[255., 255., 255.],
         [255., 255., 255.],
         [255., 255., 255.],
         ...,
         [255., 255., 255.],
         [255., 255., 255.],
         [255., 255., 255.]],

        ...,

        [[255., 255., 255.],
         [255., 255., 255.],
         [255., 255., 255.],
         ...,
         [255., 255., 255.],
         [255., 255., 255.],
         [255., 255., 255.]],

        [[255., 255., 255.],
         [255., 255., 255.],
         [255., 255., 255.],
         ...,
         [255., 255., 255.],
         [255., 255., 255.],
         [255., 255., 255.]],

        [[255., 255., 255.],
       

In [23]:
x_img_train, x_img_test = x_img_train / 255.0, x_img_test / 255.0

In [24]:
x_img_train

array([[[[1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         ...,
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ]],

        [[1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         ...,
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ]],

        [[1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         ...,
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ]],

        ...,

        [[1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ]

In [25]:
ResNet = ResNet50(weights=None, include_top=True, input_shape=(224, 224, 3), classes = y_one_hot.shape[1])

In [26]:
model = tf.keras.models.Sequential()
model.add(ResNet) 
model.add(tf.keras.layers.Flatten())

In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 983)               25601879  
                                                                 
 flatten (Flatten)           (None, 983)               0         
                                                                 
Total params: 25,601,879
Trainable params: 25,548,759
Non-trainable params: 53,120
_________________________________________________________________


In [28]:
model.compile(
    optimizer=tf.keras.optimizers.RMSprop(1e-3),
    loss=[
        tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    ],
    metrics=['accuracy']
)

In [29]:
import time
start = time.time()
print("Fit model on training data")
history = model.fit(
    x_img_train,
    y_one_hot_train,
    batch_size=20,
    epochs=10,
)
end = time.time()
print("Required training time: %f s" % (end - start))

Fit model on training data
Epoch 1/10
340/340 [==============================] - 39s 81ms/step - loss: 6.8899 - accuracy: 0.0025
Epoch 2/10
340/340 [==============================] - 27s 80ms/step - loss: 6.8884 - accuracy: 0.0044
Epoch 3/10
340/340 [==============================] - 28s 81ms/step - loss: 6.8880 - accuracy: 0.0049
Epoch 4/10
340/340 [==============================] - 28s 82ms/step - loss: 6.8873 - accuracy: 0.0052
Epoch 5/10
340/340 [==============================] - 28s 82ms/step - loss: 6.8868 - accuracy: 0.0057
Epoch 6/10
340/340 [==============================] - 28s 82ms/step - loss: 6.8868 - accuracy: 0.0056
Epoch 7/10
340/340 [==============================] - 28s 84ms/step - loss: 6.8878 - accuracy: 0.0046
Epoch 8/10
340/340 [==============================] - 28s 82ms/step - loss: 6.8878 - accuracy: 0.0046
Epoch 9/10
340/340 [==============================] - 28s 82ms/step - loss: 6.8878 - accuracy: 0.0046
Epoch 10/10
340/340 [==============================] - 

In [30]:
start = time.time()
results = model.evaluate(
    x_img_test, 
    y_one_hot_test, 
    batch_size=10)
end = time.time()
print("Required duration: %f s" % (end - start))

170/170 [==============================] - 4s 15ms/step - loss: 6.8871 - accuracy: 0.0053
Required duration: 3.831820 s


In [ ]:
from sklearn.metrics import classification_report

start = time.time()

y_pred = model.predict(x_img_test, batch_size=10, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

end = time.time()

print(classification_report(y_encoded_test, y_pred_bool))
print("Required duration: %f s" % (end - start))

In [ ]:
model.save('Resnet_only_CNN.h5')